In [ ]:
import cv2
import numpy as np
import time
import datetime

cap = cv2.VideoCapture("C:\\Users\\newxe\\Downloads\\KakaoTalk_Video_20200504_1952_13_790.mp4")

new_x = 0
move_stop = 0
count = 0
record = False
r_start = True
start = False
start_time = time.time()
move = False
flag = 0
re = 0
x = 0

def Rotate(src, degrees):
    if degrees == 90:
        dst = cv2.transpose(src)
        dst = cv2.flip(dst, 1)

    elif degrees == 180:
        dst = cv2.flip(src, -1)

    elif degrees == 270:
        dst = cv2.transpose(src)
        dst = cv2.flip(dst, 0)
    else:
        dst = null
    return dst

fgbg = cv2.createBackgroundSubtractorMOG2(history=1, varThreshold=500)
fourcc = cv2.VideoWriter_fourcc(*'XVID')


while(1):
    ret, frame = cap.read()
    
    frame = Rotate(frame, 90)
    
    if ret == False:
        break
    
    width = frame.shape[1]
    height = frame.shape[0]
    frame = cv2.resize(frame, (int(width*0.5), int(height*0.5)))

    fgmask = fgbg.apply(frame)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(fgmask)
    
    result_time = time.time() - start_time
    str = "time : %d"%result_time
    cv2.putText(frame, str, (0, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))

    for index, centroid in enumerate(centroids):
        if stats[index][0] == 0 and stats[index][1] == 0:
            continue
        if np.any(np.isnan(centroid)):
            continue


        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        if area > 10:
            flag = 1
            
    if new_x == x:
        count += 1
    else:
        count = 0
        
    new_x = x

    if count > 100:
        move_stop = 1
    
    cv2.imshow('mask',fgmask)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    elif move_stop == 1 and r_start == True:
        r_start = False
        start = True
        move_stop = 0
        record = True
        video = cv2.VideoWriter("C:\\Users\\newxe\\Downloads\\비디오 고정확인! 녹화시작" + ".avi", fourcc, 20.0, (frame.shape[1], frame.shape[0]))
        print('움직임 멈춤!! 비디오 고정확인!! 녹화시작')
    elif result_time >= 5.0 and start == False and flag == 0:
        r_start = False
        record = True
        video = cv2.VideoWriter("C:\\Users\\newxe\\Downloads\\5초 경과 녹화시작" + ".avi", fourcc, 20.0, (frame.shape[1], frame.shape[0]))
        print('5초 경과!! 움직임 없음!! 비디오 고정으로 판단!! 녹화시작!!')
        start = True
    
        
    if record == True:
        video.write(frame)    
        
cap.release()
cv2.destroyAllWindows()